# Figure 6b-d: fluorescence vs phase
This notebook pulls out some examples to use with out fluorescence model. We begin with looking at some Francisella phase vs fluorescence segmentation. Then we select some fields of view from the fluorescence test set. 

In [ ]:
# First, import dependencies.
import numpy as np
import time, os, sys

# This checks to see if you have set up your GPU properly.
# CPU performance is a lot slower, but not a problem if you are only processing a few images.
use_GPU = core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)

# for plotting 
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300

from cellpose_omni import io, transforms, plot, models, core
import omnipose
import skimage.io

modeldir = '/home/kcutler/DataDrive/omnipose_all/fluor/train_sorted/models/cellpose_residual_on_style_on_concatenation_off_omni_train_sorted_2022_04_20_13_25_40.528729_epoch_3051'


In [ ]:

files = ['/home/kcutler/DataDrive/Francisella/multi/xy3tile_14/2_phase.png',
         '/home/kcutler/DataDrive/Francisella/multi/xy3tile_14/4_green.png']

imgs = [io.imread(f) for f in files]

# print some infor about the images 
for i in imgs:
    print(i.shape)
nimg = len(imgs)
print(nimg)

plt.figure(figsize=[4]*2) # initialize figure
for k in range(len(imgs)):
    img = transforms.move_min_dim(imgs[k]) # move the channel dimension last
    if len(img.shape)>2:
        imgs[k] = img[:,:,1]
        
    imgs[k] = omnipose.utils.normalize99(imgs[k])
    plt.subplot(1,len(files),k+1)
    plt.imshow(imgs[k],cmap='gray')
    plt.axis('off')

In [ ]:
model_name = ['bact_omni','bact_fluor_omni']
model = [models.CellposeModel(gpu=use_GPU, model_type=model_name[0]),
         models.CellposeModel(gpu=use_GPU, pretrained_model=modeldir, net_avg=False, diam_mean=0, nclasses=4)]

In [ ]:
chans = [0,0] 
nimg = len(imgs)
n = range(nimg) 

# define parameters
mask_threshold = -1 
verbose = 0 # turn on if you want to see more output 
use_gpu = use_GPU #defined above
transparency = True # transparency in flow output
rescale= None # give this a number if you need to upscale or downscale your images
flow_threshold = 0 # default is .4, but only needed if there are spurious masks to clean up; slows down output
resample = True #whether or not to run dynamics on rescaled grid or original grid 
cluster = True
omni = True

N = len(model_name)
masks, flows, styles = [[]]*N, [[]]*N, [[]]*N
for i in range(N):
    masks[i], flows[i], styles[i] = model[i].eval([imgs[i]],channels=chans,rescale=rescale,mask_threshold=mask_threshold,
                                               transparency=transparency,flow_threshold=flow_threshold,omni=omni, 
                                               resample=resample,verbose=verbose, cluster=cluster,interp=True)

In [ ]:
for i in range(N):

    maski = masks[i][0]
    flowi = flows[i][0][0]
    fig = plt.figure(figsize=(12,5))
    # im = transforms.move_min_dim(imgs[i])
    # print(im.shape)
    plot.show_segmentation(fig, imgs[i], maski, flowi, channels=chans, omni=True)
    plt.tight_layout()
    plt.show()

In [ ]:
save0 = os.path.join('/home/kcutler/DataDrive/omnipose_paper/Figure 6/Francisella')
io.check_dir(save0)
clean = 1
if clean:
    for j in range(nimg):
        savedir = os.path.join(save0,model_name[j])
        io.check_dir(savedir)
        io.save_masks([imgs[j]], masks[j], flows[j], 
                      files[j], save_flows=True, save_outlines=True, 
                      savedir=savedir, in_folders=True, save_txt=False)
        io.imsave(os.path.join(savedir,'img.tif'),(imgs[j]*(2**16-1)).astype(np.uint16))

In [ ]:
# the interative widget allows us to hover and find out the index of the cell we want to grab

%matplotlib widget
plt.imshow(masks[0][0])

In [ ]:
# make outline view
%matplotlib inline
import skimage.measure
import ncolor
ext = '.png'
cellID = [229,235] #two cells in the same cluster from the two segmentation modes 

# define outline color
cmap = mpl.cm.get_cmap('plasma')
outline_col = cmap(0.85)[:3]

# define image padding
pad = 10

In [ ]:
for i in range(2):  # outler loop allows once go around to grab the crop box I want, then go again to save masks etc.
                    # by applying the second crop box to both 
    for j in range(nimg):
        labels = masks[j][0]
        bin0 = labels>0
        colonies = skimage.measure.label(bin0)
        colonyID = colonies[labels==cellID[j]]
        colony_mask = colonies==colonyID[0]

        inds = np.nonzero(colony_mask)
        max_inds = np.array(colony_mask.shape)-1
        
        slc = tuple([slice(max(0,min(inds[k])-pad),min(max_inds[k],max(inds[k])+pad)) for k in range(labels.ndim)])

        if j==1: # fluor
            slc_fluor = slc

        if i==1:
            slc = slc_fluor

        crop_img = imgs[j][slc]
        crop_masks = labels[slc]
        crop_outli = plot.outline_view(crop_img,crop_masks,color=outline_col)#,mode='thick')
    
        if i==1:
            plt.imshow(crop_outli,interpolation='none')
            plt.axis('off')
            plt.show()


            basedir = os.path.join(save0,model_name[j])
            # save the cropped image, RGB uint8 is not interpolated in illustrator ;) 
            img0 = np.stack([crop_img]*3,axis=-1)
            savepath = os.path.join(basedir,'crop_img'+ext)
            io.imsave(savepath,np.uint8(img0*(2**8-1)))

            # save the masks
            savepath = os.path.join(basedir,'crop_masks'+ext)
            io.imsave(savepath,np.uint8(crop_masks))

            #save a grayscale version for adobe illustator vectorization 
            ncl = ncolor.label(crop_masks)
            grey_n = np.stack([1-omnipose.utils.rescale(ncl)]*3,axis=-1)
            savepath = os.path.join(basedir,'masks_gray'+ext)
            io.imsave(savepath,np.uint8(grey_n*(2**8-1)))
    
        

    

### More fluorescence examples

In [ ]:
mask_filter = '_masks'
# img_names = ['/home/kcutler/DataDrive/omnipose_all/fluor/test_sorted/A22/s_flex_a22_t12xy5c1_tile1_cyto.tif',
img_names = ['/home/kcutler/DataDrive/omnipose_all/fluor/test_sorted/cex/s_flex_cex_xy5_tile1_cyto.tif',
             '/home/kcutler/DataDrive/omnipose_all/fluor/test_sorted/bthai/bthai_f_t010xy3_tile4_membrane.tif',
             '/home/kcutler/DataDrive/omnipose_all/fluor/test_sorted/bthai/bthai_f_t010xy3_tile4_cyto.tif']
mask_names,flow_names = io.get_label_files(img_names, mask_filter)
imgs = [skimage.io.imread(f) for f in img_names]
masks_gt = [skimage.io.imread(f) for f in mask_names]


In [ ]:
model_name = 'bact_fluor_omni'
model =  models.CellposeModel(gpu=use_GPU, pretrained_model=modeldir, net_avg=False, diam_mean=0, nclasses=4)

In [ ]:
chans = [0,0] 
nimg = len(imgs)
n = range(nimg) 

# define parameters
mask_threshold = -1 
verbose = 0 # turn on if you want to see more output 
use_gpu = use_GPU #defined above
transparency = True # transparency in flow output
rescale= None # give this a number if you need to upscale or downscale your images
flow_threshold = 0 # default is .4, but only needed if there are spurious masks to clean up; slows down output
resample = True #whether or not to run dynamics on rescaled grid or original grid 
cluster = True
omni = True

masks, flows, styles = model.eval(imgs,channels=chans,rescale=rescale,mask_threshold=mask_threshold,
                                               transparency=transparency,flow_threshold=flow_threshold,omni=omni, 
                                               resample=resample,verbose=verbose, cluster=cluster,interp=True)

In [ ]:
for i in range(len(imgs)):

    maski = masks[i]
    flowi = flows[i][0]
    fig = plt.figure(figsize=(12,5))
    # im = transforms.move_min_dim(imgs[i])
    # print(im.shape)
    plot.show_segmentation(fig, imgs[i], maski, flowi, channels=chans, omni=True)
    plt.tight_layout()
    plt.show()

In [ ]:
# Ths time, use coordinates; more immune to issues re-running 
%matplotlib widget
plt.figure(figsize=(4,)*2)
plt.imshow(masks[0])
plt.axis('off')
# coords = [[76,400],[180,425]]


In [ ]:
%matplotlib inline

import skimage.measure
import ncolor
ext = '.png'
save0 = os.path.join('/home/kcutler/DataDrive/omnipose_paper/Figure 6/fluor_examples')
io.check_dir(save0)
# name = ['A22','bthai_membrane','bthai_cyto']
name = ['cex','bthai_membrane','bthai_cyto']

coords = [[400,76],[282,251],[282,251]]
# coords = [[667,1713],[282,251],[282,251]]

pad = 10

cmap = mpl.cm.get_cmap('plasma')
outline_col = cmap(0.85)[:3]

nimg = len(imgs)
for i in range(2):
    for j in range(nimg):
        labels = masks[j]
        bin0 = labels>0
        colonies = skimage.measure.label(bin0)
        regs = skimage.measure.regionprops(colonies)
        centroids = np.array([r.centroid for r in regs[1:]])
        imin = np.argmin(np.sum((centroids-coords[j])**2,axis=1))
        colony_mask = colonies==regs[imin+1].label

        inds = np.nonzero(colony_mask)
        max_inds = np.array(colony_mask.shape)-1

        slc = tuple([slice(max(0,min(inds[k])-pad),min(max_inds[k],max(inds[k])+pad)) for k in range(labels.ndim)])

        if j==2: # save the cyto bbox
            slc_fluor = slc

        if i==1 and j>0: # use it on membrane second time around
            slc = slc_fluor

        if i==1:
            crop_img = omnipose.utils.normalize99(imgs[j][slc])
            crop_masks = labels[slc]
            crop_outli = plot.outline_view(crop_img,crop_masks,color=outline_col)#,mode='thick')


            plt.imshow(crop_outli,interpolation='none')
            plt.axis('off')
            plt.show()


            basedir = os.path.join(save0,name[j])
            io.check_dir(basedir)
            # save the cropped image, RGB uint8 is not interpolated in illustrator ;) 
            img0 = np.stack([crop_img,]*3,axis=-1)
            savepath = os.path.join(basedir,'crop_img'+ext)
            io.imsave(savepath,np.uint8(img0*(2**8-1)))
            
            # save the outlines
            savepath = os.path.join(basedir,'crop_outlines'+ext)
            io.imsave(savepath,np.uint8(crop_outli*(2**8-1)))
            
            # save the masks
            savepath = os.path.join(basedir,'crop_masks'+ext)
            io.imsave(savepath,np.uint8(crop_masks))
            
            # save the flows
            savepath = os.path.join(basedir,'crop_flows'+ext)
            skimage.io.imsave(savepath,np.uint8(flows[j][0][slc]))
            
            # save the distance
            savepath = os.path.join(basedir,'crop_dist'+ext)
            dist = omnipose.utils.rescale(flows[j][2][slc])
            cmap = mpl.cm.get_cmap('plasma')
            pic = cmap(dist)
            pic[:,:,-1] = crop_masks>0
            skimage.io.imsave(savepath,np.uint8(pic*(2**8-1)))
            
            # save the boundary
            savepath = os.path.join(basedir,'crop_bd'+ext)
            dist = omnipose.utils.rescale(flows[j][4][slc])
            cmap = mpl.cm.get_cmap('viridis')
            pic = cmap(dist)
            pic[:,:,-1] = crop_masks>0
            skimage.io.imsave(savepath,np.uint8(pic*(2**8-1)))

            #save a grayscale version for adobe illustator vectorization 
            ncl = ncolor.label(crop_masks)
            grey_n = np.stack([1-omnipose.utils.rescale(ncl)]*3,axis=-1)
            savepath = os.path.join(basedir,'masks_gray'+ext)
            io.imsave(savepath,np.uint8(grey_n*(2**8-1)))

            if j==2:
                membrane = omnipose.utils.normalize99(imgs[1][slc])
                img0 = np.stack([membrane,crop_img,np.zeros_like(crop_img)],axis=-1)
                savepath = os.path.join(basedir,'crop_img_rgb'+ext)
                io.imsave(savepath,np.uint8(img0*(2**8-1)))


In [ ]:
plt.imshow(flows[j][4])

In [ ]:
pic.shape

In [ ]:
plt.imshow(pic)